<a href="https://colab.research.google.com/github/varunMVP/Ewaste-Classification/blob/main/Ewaste_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Mount Google Drive (if dataset is there)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from google.colab import files
uploaded = files.upload()  # Choose your dataset.zip file


Saving E-Waste classification dataset.zip to E-Waste classification dataset.zip


In [8]:
import zipfile
with zipfile.ZipFile("E-Waste classification dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/ewaste_dataset")


In [9]:
# STEP 3: Set up ImageDataGenerator with augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = "/content/ewaste_dataset/modified-dataset/train"
val_path = "/content/ewaste_dataset/modified-dataset/val"
test_path = "/content/ewaste_dataset/modified-dataset/test"

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
).flow_from_directory(train_path, target_size=(224, 224), class_mode='categorical', batch_size=32)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path, target_size=(224, 224), class_mode='categorical', batch_size=32)
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, target_size=(224, 224), class_mode='categorical', batch_size=32, shuffle=False)


Found 2400 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


In [10]:
# STEP 4: Load Pretrained MobileNetV2
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
# STEP 5: Train the model
history = model.fit(train_gen, epochs=10, validation_data=val_gen)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6062 - loss: 1.2153

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.6084 - loss: 1.2088 - val_accuracy: 0.9367 - val_loss: 0.2230
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.9000 - loss: 0.3168 - val_accuracy: 0.9367 - val_loss: 0.2122
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.9214 - loss: 0.2257 - val_accuracy: 0.9567 - val_loss: 0.1772
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.9458 - loss: 0.1577 - val_accuracy: 0.9367 - val_loss: 0.1966
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.9598 - loss: 0.1358 - val_accuracy: 0.9467 - val_loss: 0.1762
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.9569 - loss: 0.1277 - val_accuracy: 0.9467 - val_loss: 0.1898
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.9668 - loss: 0.1122 - val_accuracy: 0.9567 - val_loss: 0.1451
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.9719 - loss: 0.0870 - val_accuracy: 0.9433 - val_loss: 0.

In [12]:
# STEP 6: Evaluate the model
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc*100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9305 - loss: 0.1911
Test Accuracy: 94.33%


In [13]:
# STEP 7: Save the model
model.save("ewaste_mobilenet_model.h5")


In [14]:
# STEP 8: Download it locally or to drive
from google.colab import files
files.download("ewaste_mobilenet_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from google.colab import files
uploaded = files.upload()  # Upload a .jpg or .png image


Saving battery_4.jpg to battery_4.jpg


In [17]:
from tensorflow.keras.models import load_model

model = load_model("ewaste_mobilenet_model.h5")


In [18]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = list(uploaded.keys())[0]

# Load and resize image to 224x224
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # normalize
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)
class_index = np.argmax(pred)

# Map index to class label
class_labels = list(train_gen.class_indices.keys())  # reuse class names from training
print(f"Predicted Class: {class_labels[class_index]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: Battery
